In [19]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [20]:
data=pd.read_csv("data/ner_dataset.csv",encoding = "ISO-8859-1")
data.shape

(1048575, 4)

In [21]:
def load_data(file_path):
    with open(file_path,"r") as file:
        data=[ line.strip() for line in file.readlines()] 
    return data
train_sentence=load_data('data/large/train/sentences.txt')
train_labels=load_data('data/large/train/labels.txt')

val_sentence=load_data('data/large/val/sentences.txt')
val_labels=load_data('data/large/val/labels.txt')
test_sentence=load_data('data/large/test/sentences.txt')
test_labels=load_data('data/large/test/labels.txt')
    

In [22]:
def create_vocab(sentences):
    vocab={}
    vocab["<unk>"]=1
    i=2
    for sentence in sentences:
        sentence=sentence.strip().split()
        for word in sentence:
            if word not in vocab.keys():
                vocab[word]=i
                i+=1
    return vocab

vocab=create_vocab(train_sentence)
label_vocab=create_vocab(train_labels)
label_vocab


{'<unk>': 1,
 'O': 2,
 'B-geo': 3,
 'B-gpe': 4,
 'B-per': 5,
 'I-geo': 6,
 'B-org': 7,
 'I-org': 8,
 'B-tim': 9,
 'B-art': 10,
 'I-art': 11,
 'I-per': 12,
 'I-gpe': 13,
 'I-tim': 14,
 'B-nat': 15,
 'B-eve': 16,
 'I-eve': 17,
 'I-nat': 18}

In [23]:
def id_to_tag(vocab):
    tag_map={}
    for i,tag in enumerate(vocab.keys()):
        tag_map[tag]=i+1
    return tag_map
tag_map=id_to_tag(label_vocab)
tag_map["<unk>"]

1

In [24]:
def vectorized_sentence(vocab,sentences):
    vectorized_sentences=[]
    max_len=0
    for i,sentence in enumerate(sentences):
        sentence=sentence.strip().split()
        max_len=np.maximum(max_len,len(sentence))
        vectorized=[vocab.get(word,1) for word in sentence]
        vectorized_sentences.append(torch.tensor(vectorized))
    return vectorized_sentences,max_len

In [25]:
train_data,max_len1=vectorized_sentence(vocab,train_sentence)
test_data,max_len2=vectorized_sentence(vocab,test_sentence)
train_label_data,_=vectorized_sentence(label_vocab,train_labels)
test_label_data,_=vectorized_sentence(label_vocab,test_labels)

In [26]:
max_len=0
max_len=np.maximum(max_len,max_len1)
max_len=np.maximum(max_len,max_len2)

In [27]:
from torch.nn.utils.rnn import pad_sequence
len_train=len(train_data)
data_for_padding=train_data+test_data
data=pad_sequence(data_for_padding,padding_value=0,batch_first=True)
X_train=data[:len_train]
X_test=data[len_train:]


In [28]:
data_for_padding2=train_label_data+test_label_data
data1=pad_sequence(data_for_padding2,padding_value=0,batch_first=True)
y_train=data1[:len_train]
y_test=data1[len_train:]


In [29]:
train_dataset=torch.utils.data.TensorDataset(X_train,y_train)
test_dataset=torch.utils.data.TensorDataset(X_test,y_test)


In [30]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=True)


In [31]:
embedding_size=64
hidden_size=128


class Ner_model(nn.Module):
    def __init__(self,vocab_size,embedding_size,output_size,hidden_size):
        super(Ner_model,self).__init__()
        self.embedding=nn.Embedding(vocab_size,embedding_size,padding_idx=0)
        self.rnn=nn.LSTM(embedding_size,hidden_size,num_layers=2,batch_first=True)
        self.out=nn.Linear(hidden_size,output_size)
    def forward(self,x):
        x=self.embedding(x)
        x,_=self.rnn(x)
        x=self.out(x)
        return x
model=Ner_model(len(vocab),embedding_size,output_size=len(label_vocab),hidden_size=hidden_size)
_,k=next(iter(train_loader))
k.shape

torch.Size([64, 104])

In [32]:
loss=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [33]:
from engine import train
device="cuda" if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [34]:
result=train(model,train_loader,test_loader,optimizer,loss,5,device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def vectorize_single_sentence(vocab,z):
    z,_=vectorized_sentence(vocab,z)
    z=z[0]
    new_z=torch.cat((z,torch.zeros(max_len-z.shape[0]))).type(torch.int64)
    new_z=new_z.unsqueeze(0)
    return new_z.to(device)

1

In [3]:
test="Iran hello European"
def predict(vocab,test):
    hi=vectorize_single_sentence(vocab,test)
    kq=model(hi)
    kq.argmax(dim=1)
    kq.squeeze(0)
    kq_char=[]
    for i in range(max_len):
        if kq[i]!=0:
            kq_char.append(tag_map[i])
    return kq_char

        

SyntaxError: incomplete input (1735618248.py, line 7)

In [ ]:
z=torch.tensor([1])
k=z.item()
type(k)

int

In [ ]:
import torch


In [ ]:
torch.cuda.is_available()

False

In [ ]:
test=nn.Embedding(10,5,padding_idx=0)
k=torch.tensor([1,2,3,0,0,0])
test(k).shape

torch.Size([6, 5])